# Module 2 Lab 3 - Generalized Estimating Equations for Repeated Categorical Measurements

In lab 2 we looked at the Repeated Measures ANOVA approach for analyzing repeated data.  One of the assumptions for ANOVARM is that we have continuous measures.  If instead, our measures are categorical in nature, then we cannot apply ANOVARM.

Generalized Estimating Equations (GEE) is an extension to the idea of Generalized Linear Models (GLM), to which class linear regression belongs.  In GLM, one of the assumptions is the independence of data.  In a study design where data are collected longitudinally, this assumption would not be true, as measurements taken from one subject at a point in time are related to other measurements taken on that subject either before or after.  So _within subject_ measurements are not independent, but _across subjects_ might be.

GEE is a model that accounts for the within-subjects relationships, and also can handle categorical (both nominal and ordinal) dependent variables, as well as continuous.  Using GEE is an approach to analysis of repeated categorical measurements that we will explore in this lab for categorical dependent variables.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
!{sys.executable} -m pip install --upgrade "statsmodels>=0.11" # 0.11 or above is required for the QIC method

from statsmodels.genmod.generalized_estimating_equations import GEE, NominalGEE
from statsmodels.genmod.cov_struct import Exchangeable, Independence, Autoregressive
from statsmodels.genmod.families import Poisson, Gaussian, Binomial

Requirement already up-to-date: statsmodels>=0.11 in /opt/conda/lib/python3.7/site-packages (0.13.1)


## Load data
We are using a dataset for a study of wheezing in children aged 7 to 10 years old in Ohio.  The primary measurement is a dichotomous variable indicating wheezing or no wheezing.  The study is longitudinal, and the status was measured once per year for fours years.

Further documentation on the file is [here](../resources/wheeze.html)


In [2]:
data = pd.read_csv('../resources/wheeze.csv', index_col=0)
data.head()

,resp,id,age,smoke
1,0,0,-2,0
2,0,0,-1,0
3,0,0,0,0
4,0,0,1,0
5,0,1,-2,0


## Preprocessing
Some simple preprocessing will help make the data easier to interpret.  In this case, converting the age variable into values that represent the age of the child rather than a relative age.  See the data documentation if the below isn't clear to you.

In [3]:
data['age'] = data['age'] + 9

## Exploratory data analysis
We will make some tables of our data to better understand it.

### Count of subjects by smoking status of mother
We can see there are more mothers who did not report smoking than did.  For regression, we do not need to be concerned about this imbalance.

In [4]:
data[data['age'] == 7].groupby(by='smoke')[['id']].count()

,id
smoke,
0,350
1,187


### Wheeze status by age
It's unclear if there is a pattern for the wheezing status over the four time points.  GEE will tell us if there is or is not.

In [5]:
pivot = data.pivot_table(values='id', index='age', columns='resp', aggfunc='count')
display(pivot)

resp,0,1
age,,
7,450,87
8,446,91
9,452,85
10,474,63


## Specifications for the GEE
When modeling the GEE, one must make a determination about how time points are related, this is known as the dependence or correlation structure.  There are a number of possible options:

1. Independent - the measurements at each time point are independent of each other.  For modeling repeated measures data, this option is not suitable because we would expect some correlation between the data at different time points
1. Exchangeable - the correlation between two time points within a subject are equal for _any_ two timepoints
1. Autoregressive - successive time points are correlated to prior time points, and the correlation weakens the farther apart the two time points are
1. Unstructured - the correlation is estimated for any two time point combinations

Understanding the data will help you choose an appropriate correlation structure, but the model will likely not fail if you choose the wrong one.  In the worst case, the structure will not converge and you will see a warning, in which case that correlation structure should not be used, and the model is invalid.

The documentation for the available correlation structures is [here](https://www.statsmodels.org/devel/gee.html#module-reference).


For this data, we will choose Exchangeable, as it's reasonable to assume that from year to year wheezing in a child may be equally correlated. Autoregressive is another potential option that can be tried, as it is also reasonable to assume that prior wheezing my be correlated to future wheezing in a child.

## Specifying the distribution

We also need to select a distribution that fits the dependent variable.  Recall from basic statistics that different types of variables follow different random distributions:

1. continuous - Normal distribution
1. binary - Bernoulli distribution
1. counts/categorical - Poisson distribution

We are primarily interested in using GEE for categorical or binary response varaibles, so we'll be choosing Bernoulli or Poisson in repeated categorical analysis.  In this specific case, our response is binary, so we choose Bernoulli.


## Specifying the formula
The formula is specified as you would a regression or GLM model, with one dependent (or endogenous) variable, and one or more independent (or exogenous) variables.

## Putting it all together
We plan to run a GEE model, using the Binomial distribution and a correlation structure that assumes Exchangeable correlation.  The dependent variable is `resp`, which is binary, and the independent variables are `age`, which is categorical, and `smoke`, which is binary.  Each subject is grouped by the `id` column, whch we will also need to specify.  

The GEE formula is thus:
`resp ~ C(age) + smoke`


In [6]:
cov_struct = Exchangeable()
model = GEE.from_formula(formula="resp ~ C(age) + C(smoke)", groups="id", data=data, cov_struct=cov_struct, family=Binomial())
result = model.fit()
print(result.summary())
print(cov_struct.summary())
print(result.scale)


                               GEE Regression Results                              
Dep. Variable:                        resp   No. Observations:                 2148
Model:                                 GEE   No. clusters:                      537
Method:                        Generalized   Min. cluster size:                   4
                      Estimating Equations   Max. cluster size:                   4
Family:                           Binomial   Mean cluster size:                 4.0
Dependence structure:         Exchangeable   Num. iterations:                     4
Date:                     Tue, 25 Jan 2022   Scale:                           1.000
Covariance type:                    robust   Time:                         12:49:50
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.7434      0.137    -12.688      0.000      -2.013      -1.47

## Interpretation of the Model

We will look at the parameter estimates of the output to help interpret the model in this table:

```
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------

```

The first column we will look at is the p value `P>|z|`.  Using an $\alpha$ = 0.05, we can see that only the value of age at time point 10 (`C(age)[T.10]`) is below the significance level.  The dichotomous variable `smoke` is somewhat close to providing significance, but not quite.  

The next column to look at is `coef`.  This tells us both the direction and the relative strength of the relationship between the variable and the outcome.  In the case of `C(age)[T.10]`, the effect is negative.  That is to say, being 10 years old reduces the probability of wheezing.

## The Quasi-likelihood under Independence Model Criterion
The QIC can be used as a relative measure of the model fit when comparing the same model and data using different correlation structures.  The QIC is the first result returned in the tuple below.  The model that returns the smallest QIC would be considered the best.

In [7]:
print(result.qic(result.scale))

(1818.7147826912203, 1827.1151606654905)


## Run the model with a different correlation structure and compare the QIC.
We will run the same model but using the Autoregressive correlation structure, since that was another choice we considered.

We will print the QIC and compare to the prior model.

In [8]:
cov_struct = Autoregressive()
model = GEE.from_formula(formula="resp ~ C(age) + C(smoke)", groups="id", data=data, cov_struct=cov_struct, family=Binomial())
result = model.fit()
print(result.summary())
print(cov_struct.summary())
print(result.scale)
print(result.qic(result.scale))

/opt/conda/lib/python3.7/site-packages/statsmodels/genmod/cov_struct.py:798: FutureWarning: grid=True will become default in a future version
  FutureWarning


                               GEE Regression Results                              
Dep. Variable:                        resp   No. Observations:                 2148
Model:                                 GEE   No. clusters:                      537
Method:                        Generalized   Min. cluster size:                   4
                      Estimating Equations   Max. cluster size:                   4
Family:                           Binomial   Mean cluster size:                 4.0
Dependence structure:       Autoregressive   Num. iterations:                     7
Date:                     Tue, 25 Jan 2022   Scale:                           1.000
Covariance type:                    robust   Time:                         12:50:00
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.7326      0.137    -12.624      0.000      -2.002      -1.46

## Interpretation
Which model is best?

The model using the Exchangeable correlation structure provided a lower QIC (by a small amount), so we would consider it the better model.  The QIC values are so close though, that the models could be considered effectively equivalent.  This shows that the intial correlation structure chosen may still lead to a good model even if the structure is not the best fit.

Ultimately, since our model failed to find significance at all of the time points, we would conclude that we cannot model a prediction for wheezing based on mother's smoking status at the start of the study, nor the incidence of wheezing over the study.  The best predictor of wheezing status was the status at the end of the study, so based on this data, we can say that smoking and prior wheezing have insignificant effects on wheezing at age 10.